In [ ]:
import json
import os
import time
from pathlib import Path
from typing import Dict, Any
from tqdm import tqdm
import logging
from langchain_openai import AzureChatOpenAI

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('json_processing.log'),
        logging.StreamHandler()
    ]
)

class ChunkProcessor:
    def __init__(self, input_dir: str, output_dir: str, batch_size: int = 5, delay: float = 1.0):
        self.input_dir = Path(input_dir)
        self.output_dir = Path(output_dir)
        self.batch_size = batch_size
        self.delay = delay
        self.output_dir.mkdir(exist_ok=True)
        
        # Initialize LLM
        self.llm = AzureChatOpenAI(
            azure_deployment="gpt-4o",
            azure_endpoint="enter ednpoint",
            openai_api_key="Enter here",
            api_version="2024-08-01-preview",
            verbose=False,
            temperature=1,
        )

    def get_summary(self, content: str) -> str:
        """Get one-line summary from LLM"""
        try:
            prompt = f"Summarize the following text in one concise line:\n\n{content}"
            response = self.llm.invoke(prompt)
            return response.content.strip()
        except Exception as e:
            logging.error(f"Error getting summary: {str(e)}")
            return None

    def process_file(self, file_path: Path):
        """Process a single JSON file"""
        #logging.info(f"Processing file: {file_path}")
        
        # Read input JSON
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        output_file = self.output_dir / file_path.name
        
        # Load existing progress if any
        existing_data = {}
        if output_file.exists():
            with open(output_file, 'r', encoding='utf-8') as f:
                existing_data = json.load(f)

        # Process chunks in batches
        chunks_to_process = []
        for chunk in data['chunks']:
            # Skip if chunk already has summary
            if (existing_data and 
                'chunks' in existing_data and 
                any(ec['id'] == chunk['id'] and 'summary' in ec for ec in existing_data['chunks'])):
                continue
            chunks_to_process.append(chunk)

        if not chunks_to_process:
            logging.info(f"All chunks in {file_path.name} already processed")
            return

        for i in range(0, len(chunks_to_process), self.batch_size):
            batch = chunks_to_process[i:i + self.batch_size]
            
            for chunk in batch:
                #logging.info(f"Processing chunk {chunk['id']}")
                summary = self.get_summary(chunk['content'])
                if summary:
                    chunk['summary'] = summary
                else:
                    logging.warning(f"Failed to get summary for chunk {chunk['id']}")

            # Update existing data with new summaries
            if not existing_data:
                existing_data = data.copy()
            
            # Save progress after each batch
            with open(output_file, 'w', encoding='utf-8') as f:
                json.dump(existing_data, f, indent=2)

            logging.info(f"Saved progress after batch. Sleeping for {self.delay} seconds...")
            time.sleep(self.delay)

    def process_all_files(self):
        """Process all JSON files in the input directory"""
        for file_path in self.input_dir.glob('*.json'):
            try:
                self.process_file(file_path)
            except Exception as e:
                logging.error(f"Error processing {file_path}: {str(e)}")
                continue



In [6]:
import os
from pathlib import Path

# Get current directory
current_dir = Path(os.getcwd())

# Create directories
input_dir = current_dir / "_NEW CHUNKS"
output_dir = current_dir / "_COMPLETE CHUNKS"

input_dir.mkdir(exist_ok=True)
output_dir.mkdir(exist_ok=True)

# Initialize processor
processor = ChunkProcessor(input_dir, output_dir, batch_size=5, delay=1.0)

# Process files
processor.process_all_files()

2025-02-18 22:20:55,496 - INFO - Processing file: C:\Users\prana\Desktop\Metadata enrichment with LLM\MetadataEnrichment-for-RAG\_NEW CHUNKS\NEW-glacier-dg-chunks.json
2025-02-18 22:20:55,591 - INFO - Processing chunk chunk_1
2025-02-18 22:20:57,483 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:20:57,519 - INFO - Processing chunk chunk_2
2025-02-18 22:20:58,562 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:20:58,565 - INFO - Processing chunk chunk_3
2025-02-18 22:20:59,692 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:20:59,704 - INFO - Processing chunk chunk_4
2025-02-18 22:21:00,816 - INFO - HTTP Request: POST https://

2025-02-18 22:21:34,348 - INFO - Processing chunk chunk_33
2025-02-18 22:21:34,902 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:21:34,904 - INFO - Processing chunk chunk_34
2025-02-18 22:21:35,477 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:21:35,480 - INFO - Processing chunk chunk_35
2025-02-18 22:21:36,008 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:21:36,154 - INFO - Saved progress after batch. Sleeping for 1.0 seconds...
2025-02-18 22:21:37,156 - INFO - Processing chunk chunk_36
2025-02-18 22:21:38,082 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-

2025-02-18 22:22:19,785 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:22:19,967 - INFO - Saved progress after batch. Sleeping for 1.0 seconds...
2025-02-18 22:22:20,969 - INFO - Processing chunk chunk_66
2025-02-18 22:22:22,251 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:22:22,254 - INFO - Processing chunk chunk_67
2025-02-18 22:22:23,442 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:22:23,444 - INFO - Processing chunk chunk_68
2025-02-18 22:22:24,270 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:

2025-02-18 22:23:00,390 - INFO - Processing chunk chunk_97
2025-02-18 22:23:01,503 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:23:01,510 - INFO - Processing chunk chunk_98
2025-02-18 22:23:02,815 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:23:02,819 - INFO - Processing chunk chunk_99
2025-02-18 22:23:04,304 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:23:04,307 - INFO - Processing chunk chunk_100
2025-02-18 22:23:05,053 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:23:05,285 - INFO - Saved prog

2025-02-18 22:23:45,227 - INFO - Processing chunk chunk_129
2025-02-18 22:23:46,219 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:23:46,221 - INFO - Processing chunk chunk_130
2025-02-18 22:23:47,081 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:23:47,195 - INFO - Saved progress after batch. Sleeping for 1.0 seconds...
2025-02-18 22:23:48,198 - INFO - Processing chunk chunk_131
2025-02-18 22:23:48,941 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-18 22:23:48,943 - INFO - Processing chunk chunk_132
2025-02-18 22:23:49,891 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?

...

2025-02-19 00:40:46,473 - INFO - Processing chunk chunk_2350
2025-02-19 00:40:48,178 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:40:48,939 - INFO - Saved progress after batch. Sleeping for 1.0 seconds...
2025-02-19 00:40:49,941 - INFO - Processing chunk chunk_2351
2025-02-19 00:40:50,980 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:40:50,982 - INFO - Processing chunk chunk_2352
2025-02-19 00:40:51,788 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:40:51,791 - INFO - Processing chunk chunk_2353
2025-02-19 00:40:52,658 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completi

2025-02-19 00:41:33,241 - INFO - Processing chunk chunk_2382
2025-02-19 00:41:34,094 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:41:34,097 - INFO - Processing chunk chunk_2383
2025-02-19 00:41:34,828 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:41:34,830 - INFO - Processing chunk chunk_2384
2025-02-19 00:41:35,489 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:41:35,490 - INFO - Processing chunk chunk_2385
2025-02-19 00:41:36,736 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:41:37,344 - INFO - Sav

2025-02-19 00:42:17,579 - INFO - Processing chunk chunk_2414
2025-02-19 00:42:18,683 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:42:18,687 - INFO - Processing chunk chunk_2415
2025-02-19 00:42:23,120 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:42:23,629 - INFO - Saved progress after batch. Sleeping for 1.0 seconds...
2025-02-19 00:42:24,631 - INFO - Processing chunk chunk_2416
2025-02-19 00:42:25,182 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:42:25,185 - INFO - Processing chunk chunk_2417
2025-02-19 00:42:26,217 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completi

2025-02-19 00:43:18,817 - INFO - Saved progress after batch. Sleeping for 1.0 seconds...
2025-02-19 00:43:19,818 - INFO - Processing chunk chunk_2446
2025-02-19 00:43:20,294 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:43:20,295 - INFO - Processing chunk chunk_2447
2025-02-19 00:43:21,532 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:43:21,535 - INFO - Processing chunk chunk_2448
2025-02-19 00:43:22,838 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:43:22,843 - INFO - Processing chunk chunk_2449
2025-02-19 00:43:29,618 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completi

2025-02-19 00:44:12,452 - INFO - Processing chunk chunk_2478
2025-02-19 00:44:13,273 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:44:13,276 - INFO - Processing chunk chunk_2479
2025-02-19 00:44:14,264 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:44:14,268 - INFO - Processing chunk chunk_2480
2025-02-19 00:44:14,915 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-08-01-preview "HTTP/1.1 200 OK"
2025-02-19 00:44:15,475 - INFO - Saved progress after batch. Sleeping for 1.0 seconds...
2025-02-19 00:44:16,477 - INFO - Processing chunk chunk_2481
2025-02-19 00:44:17,071 - INFO - HTTP Request: POST https://vareopenai.openai.azure.com/openai/deployments/gpt-4o/chat/completi

# Chunk Hierarchy Processor

In [8]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------- ----------------------------- 3.4/12.8 MB 25.2 MB/s eta 0:00:01
     ------------------------- -------------- 8.1/12.8 MB 22.9 MB/s eta 0:00:01
     ------------------------------------ -- 12.1/12.8 MB 21.6 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 21.1 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
import json
import spacy
from pathlib import Path
import logging
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

class HierarchyProcessor:
    def __init__(self, input_dir: str):
        self.input_dir = Path(input_dir)
        self.nlp = spacy.load("en_core_web_sm")
        
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s'
        )

    def get_embedding(self, text):
        doc = self.nlp(text)
        return doc.vector

    def analyze_hierarchy(self, chunks):
        embeddings = np.array([self.get_embedding(chunk['content']) for chunk in chunks])
        similarities = cosine_similarity(embeddings)

        for i, chunk in enumerate(chunks):
            chunk['parent'] = []
            chunk['children'] = []

            # Find top 3 parents (from previous chunks)
            if i > 0:
                prev_similarities = similarities[i, :i]
                top_parents = np.argsort(prev_similarities)[-3:]
                for parent_idx in top_parents:
                    if similarities[i, parent_idx] > 0.6:  # Minimum threshold
                        chunk['parent'].append(chunks[parent_idx]['id'])

            # Find top 3 children (from subsequent chunks)
            if i < len(chunks) - 1:
                next_similarities = similarities[i, i+1:]
                top_children = np.argsort(next_similarities)[-3:]
                for child_idx in top_children:
                    actual_idx = child_idx + i + 1
                    if similarities[i, actual_idx] > 0.6:
                        chunk['children'].append(chunks[actual_idx]['id'])

        return chunks

    def process_file(self, file_path):
        logging.info(f"Processing {file_path}")
        
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # Process chunks
        data['chunks'] = self.analyze_hierarchy(data['chunks'])
        
        # Save updated file
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2)

    def process_all_files(self):
        for file_path in tqdm(list(self.input_dir.glob('*.json'))):
            try:
                self.process_file(file_path)
            except Exception as e:
                logging.error(f"Error processing {file_path}: {str(e)}")


In [14]:
# Usage in Jupyter notebook:
input_dir = Path(os.getcwd()) / "_COMPLETE CHUNKS"
processor = HierarchyProcessor(input_dir)
processor.process_all_files()

100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [03:46<00:00, 56.67s/it]
